# Key Detection Result Visualizer

In [ ]:
%pip install yfiles_jupyter_graphs neo4j --quiet
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 17.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 74.1 MB/s eta 0:00:00


In [ ]:
from neo4j import GraphDatabase
from typing import Dict
from yfiles_jupyter_graphs import GraphWidget
from ipywidgets import widgets, Output

import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import numpy as np


uri      = "neo4j+s://0bbf8364.databases.neo4j.io"
user     = "reader"
password = "copenhagen"

driver = GraphDatabase.driver(uri=uri,auth=(user,password),database='neo4j')
session = driver.session(database='neo4j')



class GraphWidgetWrapper:
    def __init__(self, index):
        self.index = index
        self.out = widgets.Output()
        display(self.out)
        self.fetch_and_render()

    # Green #549343
    # Red #B53A37
    def custom_styles_mapping_image(self, index: int, item: Dict):
        columnDataURI = "data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIGhlaWdodD0iMTYiIHdpZHRoPSIxNiIgdmlld0JveD0iMCAwIDUxMiA1MTIiIGZpbGw9IiM4NkJCRDgiPjwhLS0hRm9udCBBd2Vzb21lIEZyZWUgNi41LjEgYnkgQGZvbnRhd2Vzb21lIC0gaHR0cHM6Ly9mb250YXdlc29tZS5jb20gTGljZW5zZSAtIGh0dHBzOi8vZm9udGF3ZXNvbWUuY29tL2xpY2Vuc2UvZnJlZSBDb3B5cmlnaHQgMjAyMyBGb250aWNvbnMsIEluYy4tLT48cGF0aCBkPSJNMCA5NkMwIDYwLjcgMjguNyAzMiA2NCAzMkg0NDhjMzUuMyAwIDY0IDI4LjcgNjQgNjRWNDE2YzAgMzUuMy0yOC43IDY0LTY0IDY0SDY0Yy0zNS4zIDAtNjQtMjguNy02NC02NFY5NnptNjQgNjRWNDE2SDIyNFYxNjBINjR6bTM4NCAwSDI4OFY0MTZINDQ4VjE2MHoiLz48L3N2Zz4="
        tableDataURI = "data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIGhlaWdodD0iMTYiIHdpZHRoPSIxNiIgdmlld0JveD0iMCAwIDUxMiA1MTIiIGZpbGw9IiMzMzY1OEEiPjwhLS0hRm9udCBBd2Vzb21lIEZyZWUgNi41LjEgYnkgQGZvbnRhd2Vzb21lIC0gaHR0cHM6Ly9mb250YXdlc29tZS5jb20gTGljZW5zZSAtIGh0dHBzOi8vZm9udGF3ZXNvbWUuY29tL2xpY2Vuc2UvZnJlZSBDb3B5cmlnaHQgMjAyMyBGb250aWNvbnMsIEluYy4tLT48cGF0aCBkPSJNNjQgMjU2VjE2MEgyMjR2OTZINjR6bTAgNjRIMjI0djk2SDY0VjMyMHptMjI0IDk2VjMyMEg0NDh2OTZIMjg4ek00NDggMjU2SDI4OFYxNjBINDQ4djk2ek02NCAzMkMyOC43IDMyIDAgNjAuNyAwIDk2VjQxNmMwIDM1LjMgMjguNyA2NCA2NCA2NEg0NDhjMzUuMyAwIDY0LTI4LjcgNjQtNjRWOTZjMC0zNS4zLTI4LjctNjQtNjQtNjRINjR6Ii8+PC9zdmc+"
        primaryKeyGreyDataURI = "data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIGhlaWdodD0iMTYiIHdpZHRoPSIxNiIgdmlld0JveD0iMCAwIDUxMiA1MTIiIGZpbGw9IiNDRUNFQ0QiPjwhLS0hRm9udCBBd2Vzb21lIEZyZWUgNi41LjEgYnkgQGZvbnRhd2Vzb21lIC0gaHR0cHM6Ly9mb250YXdlc29tZS5jb20gTGljZW5zZSAtIGh0dHBzOi8vZm9udGF3ZXNvbWUuY29tL2xpY2Vuc2UvZnJlZSBDb3B5cmlnaHQgMjAyMyBGb250aWNvbnMsIEluYy4tLT48cGF0aCBkPSJNMzM2IDM1MmM5Ny4yIDAgMTc2LTc4LjggMTc2LTE3NlM0MzMuMiAwIDMzNiAwUzE2MCA3OC44IDE2MCAxNzZjMCAxOC43IDIuOSAzNi44IDguMyA1My43TDcgMzkxYy00LjUgNC41LTcgMTAuNi03IDE3djgwYzAgMTMuMyAxMC43IDI0IDI0IDI0aDgwYzEzLjMgMCAyNC0xMC43IDI0LTI0VjQ0OGg0MGMxMy4zIDAgMjQtMTAuNyAyNC0yNFYzODRoNDBjNi40IDAgMTIuNS0yLjUgMTctN2wzMy4zLTMzLjNjMTYuOSA1LjQgMzUgOC4zIDUzLjcgOC4zek0zNzYgOTZhNDAgNDAgMCAxIDEgMCA4MCA0MCA0MCAwIDEgMSAwLTgweiIvPjwvc3ZnPg=="
        primaryKeyGreenDataURI = "data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIGhlaWdodD0iMTYiIHdpZHRoPSIxNiIgdmlld0JveD0iMCAwIDUxMiA1MTIiIGZpbGw9IiM1NDkzNDMiPjwhLS0hRm9udCBBd2Vzb21lIEZyZWUgNi41LjEgYnkgQGZvbnRhd2Vzb21lIC0gaHR0cHM6Ly9mb250YXdlc29tZS5jb20gTGljZW5zZSAtIGh0dHBzOi8vZm9udGF3ZXNvbWUuY29tL2xpY2Vuc2UvZnJlZSBDb3B5cmlnaHQgMjAyMyBGb250aWNvbnMsIEluYy4tLT48cGF0aCBkPSJNMzM2IDM1MmM5Ny4yIDAgMTc2LTc4LjggMTc2LTE3NlM0MzMuMiAwIDMzNiAwUzE2MCA3OC44IDE2MCAxNzZjMCAxOC43IDIuOSAzNi44IDguMyA1My43TDcgMzkxYy00LjUgNC41LTcgMTAuNi03IDE3djgwYzAgMTMuMyAxMC43IDI0IDI0IDI0aDgwYzEzLjMgMCAyNC0xMC43IDI0LTI0VjQ0OGg0MGMxMy4zIDAgMjQtMTAuNyAyNC0yNFYzODRoNDBjNi40IDAgMTIuNS0yLjUgMTctN2wzMy4zLTMzLjNjMTYuOSA1LjQgMzUgOC4zIDUzLjcgOC4zek0zNzYgOTZhNDAgNDAgMCAxIDEgMCA4MCA0MCA0MCAwIDEgMSAwLTgweiIvPjwvc3ZnPg=="
        primaryKeyRedDataURI = "data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIGhlaWdodD0iMTYiIHdpZHRoPSIxNiIgdmlld0JveD0iMCAwIDUxMiA1MTIiIGZpbGw9IiNCNTNBMzciPjwhLS0hRm9udCBBd2Vzb21lIEZyZWUgNi41LjEgYnkgQGZvbnRhd2Vzb21lIC0gaHR0cHM6Ly9mb250YXdlc29tZS5jb20gTGljZW5zZSAtIGh0dHBzOi8vZm9udGF3ZXNvbWUuY29tL2xpY2Vuc2UvZnJlZSBDb3B5cmlnaHQgMjAyMyBGb250aWNvbnMsIEluYy4tLT48cGF0aCBkPSJNMzM2IDM1MmM5Ny4yIDAgMTc2LTc4LjggMTc2LTE3NlM0MzMuMiAwIDMzNiAwUzE2MCA3OC44IDE2MCAxNzZjMCAxOC43IDIuOSAzNi44IDguMyA1My43TDcgMzkxYy00LjUgNC41LTcgMTAuNi03IDE3djgwYzAgMTMuMyAxMC43IDI0IDI0IDI0aDgwYzEzLjMgMCAyNC0xMC43IDI0LTI0VjQ0OGg0MGMxMy4zIDAgMjQtMTAuNyAyNC0yNFYzODRoNDBjNi40IDAgMTIuNS0yLjUgMTctN2wzMy4zLTMzLjNjMTYuOSA1LjQgMzUgOC4zIDUzLjcgOC4zek0zNzYgOTZhNDAgNDAgMCAxIDEgMCA4MCA0MCA0MCAwIDEgMSAwLTgweiIvPjwvc3ZnPg=="
        foreignKeyGreyDataURI = "data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIGhlaWdodD0iMTYiIHdpZHRoPSIyMCIgdmlld0JveD0iMCAwIDY0MCA1MTIiIGZpbGw9IiNDRUNFQ0QiPjwhLS0hRm9udCBBd2Vzb21lIEZyZWUgNi41LjEgYnkgQGZvbnRhd2Vzb21lIC0gaHR0cHM6Ly9mb250YXdlc29tZS5jb20gTGljZW5zZSAtIGh0dHBzOi8vZm9udGF3ZXNvbWUuY29tL2xpY2Vuc2UvZnJlZSBDb3B5cmlnaHQgMjAyMyBGb250aWNvbnMsIEluYy4tLT48cGF0aCBkPSJNNTc5LjggMjY3LjdjNTYuNS01Ni41IDU2LjUtMTQ4IDAtMjA0LjVjLTUwLTUwLTEyOC44LTU2LjUtMTg2LjMtMTUuNGwtMS42IDEuMWMtMTQuNCAxMC4zLTE3LjcgMzAuMy03LjQgNDQuNnMzMC4zIDE3LjcgNDQuNiA3LjRsMS42LTEuMWMzMi4xLTIyLjkgNzYtMTkuMyAxMDMuOCA4LjZjMzEuNSAzMS41IDMxLjUgODIuNSAwIDExNEw0MjIuMyAzMzQuOGMtMzEuNSAzMS41LTgyLjUgMzEuNS0xMTQgMGMtMjcuOS0yNy45LTMxLjUtNzEuOC04LjYtMTAzLjhsMS4xLTEuNmMxMC4zLTE0LjQgNi45LTM0LjQtNy40LTQ0LjZzLTM0LjQtNi45LTQ0LjYgNy40bC0xLjEgMS42QzIwNi41IDI1MS4yIDIxMyAzMzAgMjYzIDM4MGM1Ni41IDU2LjUgMTQ4IDU2LjUgMjA0LjUgMEw1NzkuOCAyNjcuN3pNNjAuMiAyNDQuM2MtNTYuNSA1Ni41LTU2LjUgMTQ4IDAgMjA0LjVjNTAgNTAgMTI4LjggNTYuNSAxODYuMyAxNS40bDEuNi0xLjFjMTQuNC0xMC4zIDE3LjctMzAuMyA3LjQtNDQuNnMtMzAuMy0xNy43LTQ0LjYtNy40bC0xLjYgMS4xYy0zMi4xIDIyLjktNzYgMTkuMy0xMDMuOC04LjZDNzQgMzcyIDc0IDMyMSAxMDUuNSAyODkuNUwyMTcuNyAxNzcuMmMzMS41LTMxLjUgODIuNS0zMS41IDExNCAwYzI3LjkgMjcuOSAzMS41IDcxLjggOC42IDEwMy45bC0xLjEgMS42Yy0xMC4zIDE0LjQtNi45IDM0LjQgNy40IDQ0LjZzMzQuNCA2LjkgNDQuNi03LjRsMS4xLTEuNkM0MzMuNSAyNjAuOCA0MjcgMTgyIDM3NyAxMzJjLTU2LjUtNTYuNS0xNDgtNTYuNS0yMDQuNSAwTDYwLjIgMjQ0LjN6Ii8+PC9zdmc+"
        foreignKeyGreenDataURI = "data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIGhlaWdodD0iMTYiIHdpZHRoPSIyMCIgdmlld0JveD0iMCAwIDY0MCA1MTIiIGZpbGw9IiM1NDkzNDMiPjwhLS0hRm9udCBBd2Vzb21lIEZyZWUgNi41LjEgYnkgQGZvbnRhd2Vzb21lIC0gaHR0cHM6Ly9mb250YXdlc29tZS5jb20gTGljZW5zZSAtIGh0dHBzOi8vZm9udGF3ZXNvbWUuY29tL2xpY2Vuc2UvZnJlZSBDb3B5cmlnaHQgMjAyMyBGb250aWNvbnMsIEluYy4tLT48cGF0aCBkPSJNNTc5LjggMjY3LjdjNTYuNS01Ni41IDU2LjUtMTQ4IDAtMjA0LjVjLTUwLTUwLTEyOC44LTU2LjUtMTg2LjMtMTUuNGwtMS42IDEuMWMtMTQuNCAxMC4zLTE3LjcgMzAuMy03LjQgNDQuNnMzMC4zIDE3LjcgNDQuNiA3LjRsMS42LTEuMWMzMi4xLTIyLjkgNzYtMTkuMyAxMDMuOCA4LjZjMzEuNSAzMS41IDMxLjUgODIuNSAwIDExNEw0MjIuMyAzMzQuOGMtMzEuNSAzMS41LTgyLjUgMzEuNS0xMTQgMGMtMjcuOS0yNy45LTMxLjUtNzEuOC04LjYtMTAzLjhsMS4xLTEuNmMxMC4zLTE0LjQgNi45LTM0LjQtNy40LTQ0LjZzLTM0LjQtNi45LTQ0LjYgNy40bC0xLjEgMS42QzIwNi41IDI1MS4yIDIxMyAzMzAgMjYzIDM4MGM1Ni41IDU2LjUgMTQ4IDU2LjUgMjA0LjUgMEw1NzkuOCAyNjcuN3pNNjAuMiAyNDQuM2MtNTYuNSA1Ni41LTU2LjUgMTQ4IDAgMjA0LjVjNTAgNTAgMTI4LjggNTYuNSAxODYuMyAxNS40bDEuNi0xLjFjMTQuNC0xMC4zIDE3LjctMzAuMyA3LjQtNDQuNnMtMzAuMy0xNy43LTQ0LjYtNy40bC0xLjYgMS4xYy0zMi4xIDIyLjktNzYgMTkuMy0xMDMuOC04LjZDNzQgMzcyIDc0IDMyMSAxMDUuNSAyODkuNUwyMTcuNyAxNzcuMmMzMS41LTMxLjUgODIuNS0zMS41IDExNCAwYzI3LjkgMjcuOSAzMS41IDcxLjggOC42IDEwMy45bC0xLjEgMS42Yy0xMC4zIDE0LjQtNi45IDM0LjQgNy40IDQ0LjZzMzQuNCA2LjkgNDQuNi03LjRsMS4xLTEuNkM0MzMuNSAyNjAuOCA0MjcgMTgyIDM3NyAxMzJjLTU2LjUtNTYuNS0xNDgtNTYuNS0yMDQuNSAwTDYwLjIgMjQ0LjN6Ii8+PC9zdmc+"
        foreignKeyRedDataURI = "data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIGhlaWdodD0iMTYiIHdpZHRoPSIyMCIgdmlld0JveD0iMCAwIDY0MCA1MTIiIGZpbGw9IiNCNTNBMzciPjwhLS0hRm9udCBBd2Vzb21lIEZyZWUgNi41LjEgYnkgQGZvbnRhd2Vzb21lIC0gaHR0cHM6Ly9mb250YXdlc29tZS5jb20gTGljZW5zZSAtIGh0dHBzOi8vZm9udGF3ZXNvbWUuY29tL2xpY2Vuc2UvZnJlZSBDb3B5cmlnaHQgMjAyMyBGb250aWNvbnMsIEluYy4tLT48cGF0aCBkPSJNNTc5LjggMjY3LjdjNTYuNS01Ni41IDU2LjUtMTQ4IDAtMjA0LjVjLTUwLTUwLTEyOC44LTU2LjUtMTg2LjMtMTUuNGwtMS42IDEuMWMtMTQuNCAxMC4zLTE3LjcgMzAuMy03LjQgNDQuNnMzMC4zIDE3LjcgNDQuNiA3LjRsMS42LTEuMWMzMi4xLTIyLjkgNzYtMTkuMyAxMDMuOCA4LjZjMzEuNSAzMS41IDMxLjUgODIuNSAwIDExNEw0MjIuMyAzMzQuOGMtMzEuNSAzMS41LTgyLjUgMzEuNS0xMTQgMGMtMjcuOS0yNy45LTMxLjUtNzEuOC04LjYtMTAzLjhsMS4xLTEuNmMxMC4zLTE0LjQgNi45LTM0LjQtNy40LTQ0LjZzLTM0LjQtNi45LTQ0LjYgNy40bC0xLjEgMS42QzIwNi41IDI1MS4yIDIxMyAzMzAgMjYzIDM4MGM1Ni41IDU2LjUgMTQ4IDU2LjUgMjA0LjUgMEw1NzkuOCAyNjcuN3pNNjAuMiAyNDQuM2MtNTYuNSA1Ni41LTU2LjUgMTQ4IDAgMjA0LjVjNTAgNTAgMTI4LjggNTYuNSAxODYuMyAxNS40bDEuNi0xLjFjMTQuNC0xMC4zIDE3LjctMzAuMyA3LjQtNDQuNnMtMzAuMy0xNy43LTQ0LjYtNy40bC0xLjYgMS4xYy0zMi4xIDIyLjktNzYgMTkuMy0xMDMuOC04LjZDNzQgMzcyIDc0IDMyMSAxMDUuNSAyODkuNUwyMTcuNyAxNzcuMmMzMS41LTMxLjUgODIuNS0zMS41IDExNCAwYzI3LjkgMjcuOSAzMS41IDcxLjggOC42IDEwMy45bC0xLjEgMS42Yy0xMC4zIDE0LjQtNi45IDM0LjQgNy40IDQ0LjZzMzQuNCA2LjkgNDQuNi03LjRsMS4xLTEuNkM0MzMuNSAyNjAuOCA0MjcgMTgyIDM3NyAxMzJjLTU2LjUtNTYuNS0xNDgtNTYuNS0yMDQuNSAwTDYwLjIgMjQ0LjN6Ii8+PC9zdmc+"

        labelToIcon = {
            'Order': 'https://em-content.zobj.net/source/apple/391/basket_1f9fa.png',
            'Customer': tableDataURI,
            'Product': primaryKeyGreyDataURI,
            'Factory': foreignKeyGreyDataURI,
            'Headquarter': foreignKeyGreyDataURI
          }
        icon = labelToIcon[item['properties']['label']]

        return {
                'image': icon
        }

    def custom_node_label_mapping(self, index, node):
        properties = node.get('properties', {})
        return properties.get('id', properties.get('name', ''));

    def custom_relationship_label_mapping(self, index, node):
        properties = node.get('properties', {})
        return properties.get('label', ['']);


    def fetch_and_render(self):
        result = session.run("""
                    MATCH (n)-[e]->(m)
                    RETURN n,e,m
                    LIMIT 20
                    """, parameters={})
        w = GraphWidget(graph=result.graph())
        w.node_label_mapping = self.custom_node_label_mapping
        w.edge_label_mapping = self.custom_relationship_label_mapping
        w.set_node_styles_mapping(self.custom_styles_mapping_image)
        w.organic_layout()
        w.set_sidebar(enabled=False)
        self.out.clear_output()
        with self.out:
            w.show()


graph = GraphWidgetWrapper(1)

Output()